In [2]:
import pandas
import numpy as np

def code_variance(dataframecolumn):
    mean = dataframecolumn.mean()
    squaredsum = 0
    n = 0
    for row in dataframecolumn:
        squaredsum += (row - mean) ** 2
        n += 1
    variance = round((squaredsum / n)**(1/2.0), 2)
    if n == 0:
        return np.NaN
    else:
        return variance

CMS_In_2014 = pandas.read_csv("CMS Data/Medicare_Provider_Charge_Inpatient_DRGALL_FY2014.csv")     ###Data from cms.gov

print list(CMS_In_2014)

Inpatient_Code_List = []
CMS_In_2014sorted = CMS_In_2014.sort_values('DRG Definition')
for item in CMS_In_2014sorted['DRG Definition']:
    if item not in Inpatient_Code_List:
        Inpatient_Code_List.append(item.replace("'",""))
        
Inpatient_Mean_List_by_code = []
Inpatient_Median_List_by_code = []
Inpatient_Variance_List_by_code = []
Inpatient_N_List_by_code = []
for item in Inpatient_Code_List:
    newdf = CMS_In_2014sorted.loc[CMS_In_2014sorted['DRG Definition'] == item]
    Inpatient_Mean_List_by_code.append(round(newdf['Average Total Payments'].mean(),2))
    Inpatient_Median_List_by_code.append(round(newdf['Average Total Payments'].median(),2))
    Inpatient_Variance_List_by_code.append(code_variance(newdf['Average Total Payments']))
    Inpatient_N_List_by_code.append(newdf['Average Total Payments'].count())

dfinput = zip(Inpatient_Code_List,Inpatient_Mean_List_by_code,Inpatient_Median_List_by_code,Inpatient_Variance_List_by_code,Inpatient_N_List_by_code)

INcodesstats_National = pandas.DataFrame(data=dfinput,columns=['DRG Definition','Mean','Median','Variance','N'])

INcodesstats_National['DRG Definition'] = Inpatient_Code_List

INcodesstats_National.to_pickle('Inpatient Code Stats (CMS) - National(Fixed).pkl')

print INcodesstats_National.sort_values('Variance', ascending=False).head()

['DRG Definition', 'Provider Id', 'Provider Name', 'Provider Street Address', 'Provider City', 'Provider State', 'Provider Zip Code', 'Hospital Referral Region (HRR) Description', 'Total Discharges', 'Average Covered Charges', 'Average Total Payments', 'Average Medicare Payments']


                                        DRG Definition       Mean     Median  \
0    001 - HEART TRANSPLANT OR IMPLANT OF HEART ASS...  244554.59  235529.59   
2    003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FA...  134199.65  124844.83   
4    005 - LIVER TRANSPLANT W MCC OR INTESTINAL TRA...  125107.63  117872.75   
471                        813 - COAGULATION DISORDERS   28490.18   15238.77   
52    073 - CRANIAL & PERIPHERAL NERVE DISORDERS W MCC   13670.40    9859.89   

     Variance    N  
0    55655.38   68  
2    43433.03  454  
4    41503.21   40  
471  35526.38  116  
52   34874.77  148  


In [7]:
CMS_Out_2014 = pandas.read_csv("CMS Data/Medicare_Provider_Charge_Outpatient_APC32_CY2014.csv") 

Outpatient_Code_list = []
CMS_Out_2014sorted = CMS_Out_2014.sort_values('apc')
for item in CMS_Out_2014sorted.apc:
    if item not in Outpatient_Code_list:
        Outpatient_Code_list.append(item)

OUTcodesstats_National = pandas.DataFrame(index=[Outpatient_Code_list], columns=['Mean','Median','Variance', 'N'])

for code in OUTcodesstats_National.index:
     OUTcodesstats_National.set_value([code],['Mean'],CMS_Out_2014.loc[CMS_Out_2014['apc'] == code]['Average_Estimated_Submitted_Charges'].mean())
     OUTcodesstats_National.set_value([code],['Median'],CMS_Out_2014.loc[CMS_Out_2014['apc'] == code]['Average_Estimated_Submitted_Charges'].median()) 
     OUTcodesstats_National.set_value([code],['Variance'],code_variance(CMS_Out_2014.loc[CMS_Out_2014['apc'] == code]['Average_Estimated_Submitted_Charges']))
     OUTcodesstats_National.set_value([code],['N'],CMS_Out_2014.loc[CMS_Out_2014['apc'] == code]['Average_Estimated_Submitted_Charges'].count()) 
     
#OUTcodesstats_National.to_pickle('Outpatient Code Stats (CMS) - National.pkl')     

print OUTcodesstats_National.sort_values('Variance', ascending=False)

                                                       Mean   Median Variance  \
0074 - Level IV Endoscopy Upper Airway              10379.4  8711.36  5753.21   
0203 - Level IV Nerve Injections                    5735.32  5000.11  3549.69   
0377 - Level II Cardiac Imaging                     7115.07  6412.13  3445.29   
0073 - Level III Endoscopy Upper Airway              4106.2  3250.31  3370.34   
0020 - Level II Excision/ Biopsy                    4466.89  3773.66   3200.3   
0017 - Level V Debridement & Destruction            5895.37  5365.54  2756.66   
0207 - Level III Nerve Injections                   3181.04  2486.77  2409.38   
0270 - Level III Echocardiogram Without Contrast    3763.74  3277.84  2122.44   
0019 - Level I Excision/ Biopsy                     1505.13  834.117  1993.92   
0699 - Level IV Eye Tests & Treatments              6414.25  6384.83  1940.27   
0209 - Level II Extended EEG, Sleep, and Cardio...  2781.45  2332.02  1929.16   
0206 - Level II Nerve Inject

In [3]:
Inpatientdata = pandas.read_pickle('My Files/Parsed Data/Inpatient Code Stats (CMS) - National.pkl')
Outpatientdata = pandas.read_pickle('My Files/Parsed Data/Outpatient Code Stats (CMS) - National.pkl')

InpatientdatabyVar0 = Inpatientdata.sort_values('Variance')
OutpatientdatabyVar0 = Outpatientdata.sort_values('Variance')

InpatientdatabyVar = InpatientdatabyVar0.dropna(subset=['Variance'])
OutpatientdatabyVar = OutpatientdatabyVar0.dropna(subset=['Variance'])

from matplotlib import pyplot as plt

Infullcode = list(InpatientdatabyVar['DRG Definition'])

print Infullcode[-3:]

Inx = []

for code in Infullcode:
    Inx.append(code[:3])

Iny = list(InpatientdatabyVar.Variance)

xval = range(0,len(Inx))

plt.figure(1, figsize=(45,10), dpi=200)
plt.bar(xval, Iny, width=0.7, align='center')
plt.xticks(xval, Inx, size='xx-small', rotation='vertical')
plt.xlabel('Billable Code')
plt.ylabel('Variance')
plt.title('National Inpatient Procedure Code Price Variance')
plt.tight_layout()


#plt.savefig('National Inpatient Procedure Code Price Variance.png')

['005 - LIVER TRANSPLANT W MCC OR INTESTINAL TRANSPLANT', '003 - ECMO OR TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NECK W MAJ O.R.', '001 - HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM W MCC']


In [5]:
InN = len(InpatientdatabyVar)
OutN = len(OutpatientdatabyVar)

InMedianVal = (InN/2.0)
OutMedianVal = OutN/2.0

InLowerQ1 = [InN*0.25, InN*0.25 - 1]
InUpperQ3 = [InN*0.75, InN*0.75 + 1]

InLowerQ1Val = (InpatientdatabyVar['Variance'][int(InLowerQ1[0])]+InpatientdatabyVar['Variance'][int(InLowerQ1[1])])/2.0

InUpperQ3Val = (InpatientdatabyVar['Variance'][int(InUpperQ3[0])]+InpatientdatabyVar['Variance'][int(InUpperQ3[1])])/2.0

InMedian = InpatientdatabyVar['Variance'].median()
OutMedian = OutpatientdatabyVar['Variance'].median()

InIQRange = InUpperQ3Val - InLowerQ1Val

InQInnerFenceVal = InIQRange * 1.5
InQOuterFenceVal = InIQRange * 3

InInnerFences = [round(InLowerQ1Val-InQInnerFenceVal, 2), round(InUpperQ3Val+InQInnerFenceVal, 2)]
InOuterFences = [round(InLowerQ1Val-InQOuterFenceVal, 2), round(InUpperQ3Val+InQOuterFenceVal, 2)]

print InInnerFences, InOuterFences #These are my discriminators for finding major and minor outliers. We'll only eliminate the Major outliers.

[-3577.92, 12050.9] [-9438.73, 17911.71]


In [76]:
plt.gcf().clear() #Clear the matplotlib entries for the other plot

InpatientdatabyVarNoMajorOuts = InpatientdatabyVar[InpatientdatabyVar.Variance <= 17912]

InfullcodeNoMajorOuts = list(InpatientdatabyVarNoMajorOuts['DRG Definition'])

print InfullcodeNoMajorOuts[-3:]

Inx1 = []

for code in InfullcodeNoMajorOuts:
    Inx1.append(code[:3])

Iny1 = list(InpatientdatabyVarNoMajorOuts.Variance)

xval1 = range(0,len(Inx1))

plt.figure(2, figsize=(45,10), dpi=200)
plt.bar(xval1, Iny1, width=0.7, color='g', align='center')
plt.xticks(xval1, Inx1, size='xx-small', rotation='vertical')
plt.xlabel('Billable Code')
plt.ylabel('Variance')
plt.title('National Inpatient Procedure Code Price Variance With No Major Outliers')
plt.tight_layout()

#plt.savefig('National Inpatient Procedure Code Price Variance With No Major Outliers.png')

['463 - WND DEBRID & SKN GRFT EXC HAND, FOR MUSCULO-CONN TISS DIS W MCC', '837 - CHEMO W ACUTE LEUKEMIA AS SDX OR W HIGH DOSE CHEMO AGENT W MCC', '006 - LIVER TRANSPLANT W/O MCC']


In [77]:
plt.gcf().clear()

InpatientdatabyVarNoOuts = InpatientdatabyVar[InpatientdatabyVar.Variance <= InInnerFences[1]]

InfullcodeNoOuts = list(InpatientdatabyVarNoOuts['DRG Definition'])

print InfullcodeNoOuts[-3:]

Inx2 = []

for code in InfullcodeNoOuts:
    Inx2.append(code[:3])

Iny2 = list(InpatientdatabyVarNoOuts.Variance)

xval2 = range(0,len(Inx2))

plt.figure(3, figsize=(45,10), dpi=200)
plt.bar(xval2, Iny2, width=0.7, color='y', align='center')
plt.xticks(xval2, Inx2, size='xx-small', rotation='vertical')
plt.xlabel('Billable Code')
plt.ylabel('Variance')
plt.title('National Inpatient Procedure Code Price Variance With No Outliers')
plt.tight_layout()

#plt.savefig('National Inpatient Procedure Code Price Variance With No Outliers.png')

['459 - SPINAL FUSION EXCEPT CERVICAL W MCC', '870 - SEPTICEMIA OR SEVERE SEPSIS W MV 96+ HOURS', '928 - FULL THICKNESS BURN W SKIN GRAFT OR INHAL INJ W CC/MCC']


In [4]:
#print InpatientdatabyVar.tail()

def complicationchecker(DRGorAPCstring):
    stringlist = DRGorAPCstring
    if "W CC" in stringlist and "MCC" not in stringlist:
        return 1
    elif "W CC/MCC" in stringlist:
        return 2
    elif "W MCC" in stringlist:
        return 3
    else:
        return 0

#print InpatientdatabyVar['DRG Definition'].apply(str.split).tail()
InpatientdatabyVar['Complication Score'] = InpatientdatabyVar['DRG Definition'].apply(complicationchecker)

print InpatientdatabyVar['Complication Score'].head()

416    0
330    2
459    1
331    0
457    2
Name: Complication Score, dtype: int64


In [69]:
nozeroInpatientdatabyVar = InpatientdatabyVar[InpatientdatabyVar.Variance != 0]

nozeroInpatientdatabyVar.reset_index(drop=True, inplace=True)

mask1 = nozeroInpatientdatabyVar.loc[nozeroInpatientdatabyVar['Complication Score'] == 0]
mask2 = nozeroInpatientdatabyVar.loc[nozeroInpatientdatabyVar['Complication Score'] == 1]
mask3 = nozeroInpatientdatabyVar.loc[nozeroInpatientdatabyVar['Complication Score'] == 2]
mask4 = nozeroInpatientdatabyVar.loc[nozeroInpatientdatabyVar['Complication Score'] == 3]

mask1Inx = []
mask2Inx = []
mask3Inx = []
mask4Inx = []

for code in mask1['DRG Definition']:
    mask1Inx.append(code)
for code in mask2['DRG Definition']:
    mask2Inx.append(code)
for code in mask3['DRG Definition']:
    mask3Inx.append(code)
for code in mask4['DRG Definition']:
    mask4Inx.append(code)
    
#print mask1Inx,mask2Inx,mask3Inx,mask4Inx

mask1Iny = list(mask1.Variance)
mask2Iny = list(mask2.Variance)
mask3Iny = list(mask3.Variance)
mask4Iny = list(mask4.Variance)

boxlist = [mask1Iny, mask2Iny, mask3Iny, mask4Iny]

Totalx = len(nozeroInpatientdatabyVar['DRG Definition'])
mask1xticks = len(mask1['DRG Definition'])


labels = ['No Complications \n \n N = '+str(len(mask1Inx)),'With Complication \n \n N = '+str(len(mask2Inx)),'With Complication or \n Major Complication \n N = '+str(len(mask3Inx)),'With Major Complication \n \n N = '+str(len(mask4Inx))]

plt.gcf().clear()
plt.figure(4, figsize=[6,5], dpi=150)
plt.boxplot(boxlist, widths=0.5)
plt.xticks([1,2,3,4],labels, fontsize='x-small')
plt.ylabel('Variance (Unit-less)')
plt.xlabel('DRG Codes Grouped by Presence of Complication')
plt.title('DRG Code Price Variance \n By Presence of Complication - National')
plt.tight_layout()
# plt.show()
plt.savefig('DRG Code Price Variance, Grouped by Complication Status - National.png')